In [20]:
#!pip install mesa seaborn --quiet

In [21]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model 

# Debido a que necesitamos que existe un solo agente por celda, elegimos ''SingleGrid''.
from mesa.space import SingleGrid

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import SimultaneousActivation
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

import random

In [22]:
class LuigiAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.points = 4

    def get_adjacent_agents(self):
        # Obtener los agentes en las celdas adyacentes
        adjacent_agents = []
        possible_moves = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)

        for pos in possible_moves:
            adjacent_agents.extend(self.model.grid.get_cell_list_contents([pos]))
        
        return adjacent_agents

    def decide_to_save_points(self):
        # Decidir aleatoriamente si ahorrar puntos
        return random.choice([True, False])  # 50% de probabilidad de ahorrar

    def interact(self):
        adjacent_agents = self.get_adjacent_agents()
        
        # Buscar fantasmas
        for ghost in self.model.ghosts:
            if ghost["pos"] == self.pos and not ghost["is_dead"]:
                # Decidir cómo atacar basado en puntos
                if self.points >= 2:
                    self.perform_exorcism(ghost)
                elif self.points >= 1:
                    self.perform_knocked(ghost)
                break  # Solo interactuar con un fantasma a la vez

        # Buscar retratos
        for portrait in self.model.portraits:
            if portrait["pos"] == self.pos and not portrait["is_revealed"]:
                # Decidir si recoger el retrato basado en puntos
                if self.points >= 1:
                    self.collect_portrait(portrait)
                    break  # Solo interactuar con un retrato a la vez

    def move(self):
        if self.points > 0:
            possible_moves = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
            new_position = self.random.choice(possible_moves)
            self.model.grid.move_agent(self, new_position)
            self.points -= 1  # Costo de moverse

    def perform_knocked(self, ghost):
        if self.points >= 1:  # Costo de noquear
            ghost["points"] -= 1 # Costo para el fantasma de ser noqueado
            self.points -= 1
            print(f"Luigi {self.unique_id} ha noqueado un fantasma en {self.pos}")
            self.check_ghost_health(ghost)

    def perform_exorcism(self, ghost):
        if self.points >= 2 and ghost["points"] == 2:  # Costo de exorcizar
            ghost["points"] -= 2 # Costo para el fantasma de ser absorbido
            ghost["is_dead"] = True # Inmediatamente mata al fantasma
            self.points -= 2
            print(f"Luigi {self.unique_id} ha absorbido un fantasma en {self.pos}")

    # Revisar si un fantasma noqueado ha muerto
    def check_ghost_health(self, ghost):
        if ghost["points"] <= 0:
            ghost["is_dead"] = True
            print(f"El fantasma en {ghost['pos']} ha sido eliminado.")

    def collect_portrait(self, portrait):
        portrait["is_revealed"] = True
        self.points -= 1
        if portrait["is_fake"]:
            print(f"El retrato en {self.pos} ha sido revelado como un espejo (falsa alarma)")
        else:
            print(f"El retrato en {self.pos} ha sido revelado como un personaje real")

    def step(self):
        if not self.decide_to_save_points():
            self.move()  # Si no decide ahorrar, se mueve
        self.interact()  # Realiza interacciones

In [23]:
def get_grid(model):
    # Inicializamos una cuadrícula vacía
    grid = np.empty((model.grid.width, model.grid.height), dtype=object)
    
    # Iteramos sobre todas las celdas de la cuadrícula
    for x in range(model.grid.width):
        for y in range(model.grid.height):
            # Obtenemos los agentes en la celda actual
            agents_in_cell = model.grid.get_cell_list_contents([(x, y)])
            
            # Asignamos una lista de agentes en esa celda
            grid[x, y] = agents_in_cell
            
    return grid

In [ ]:
class MansionModel(Model):
    def __init__(self, width, height, num_agents=5, max_energy=10):
        super().__init__()
        self.grid = SingleGrid(width, height, torus=False)
        self.schedule = RandomActivation(self)
        self.ghosts = [] # Manejar fantasmas dentro del modelo
        self.portraits = [] # Manejar retratos dentro del modelo
        self.datacollector = DataCollector(
            model_reporters={
                "Grid": get_grid,
                "Steps": lambda model: model.steps,
            },
        )
        self.num_agents = num_agents  # Número de agentes fantasmas y retratos
        self.max_energy = max_energy
        self.create_agents()

        self.steps = 0

    def create_agents(self):
        # Crear fantasmas y retratos directamente dentro del modelo
        for i in range(self.num_agents):
            # Fantasmas
            ghost_data = {
                "points": 1, # Los fantasmas empiezan como Green Ghosts
                "is_dead": False, # Los fantasmas empiezan vivos
                "pos": (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height))
            }
            self.ghosts.append(ghost_data)

            # Retratos
            portrait_data = {
                "is_revealed": False, # Todos empiezan sin descubrir
                "is_fake": self.random.choice([True, False]),  # Ciertos son reales, otros no
                "damaged": False, # Todos empiezan sin daños
                "pos": (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height))
            }
            self.portraits.append(portrait_data)

        # Crear Luigi
        luigi_pos = (self.random.randrange(self.grid.width), self.random.randrange(self.grid.height))
        luigi = LuigiAgent(self.next_id(), self)
        self.grid.place_agent(luigi, luigi_pos)
        self.schedule.add(luigi)

    def get_grid(self, model):
        # Devuelve una representación de la cuadrícula
        grid = np.zeros((model.grid.width, model.grid.height), dtype=object)
        
        for cell in range(model.grid.width):
            for row in range(model.grid.height):
                cell_content = model.grid.get_cell_list_contents([(cell, row)])
                grid[cell][row] = cell_content  # Guarda los agentes de la celda
        return grid

    def step(self):
    # Realiza un paso en la simulación, actualizando el estado de todos los agentes
        for cell in self.grid.coord_iter():
            cell_agents = self.grid.get_cell_list_contents([cell[1]])  # Obtener agentes en todas las celdas
            for agent in cell_agents:
                if isinstance(agent, LuigiAgent):
                    agent.step() # Los agentes Luigi realizan su acción
        
        # Manejar lógica de los fantasmas para cada turno
        for ghost in self.ghosts:
            self.handle_ghost(ghost)

        # Manejar lógica para los retratos para cada turno
        for portrait in self.portraits:
            self.handle_portrait(portrait)

        self.steps += 1 # Se terminó un turno
        self.datacollector.collect(self)

    def handle_ghost(self, ghost):
        # Lógica para manejar los fantasmas dentro del modelo
        if not ghost["is_dead"]:
            # Si el fanstasma termina el turno con un punto, subir de nivel
            # Empezar a partir del segundo turno del juego
            if ghost["points"] == 1 and self.steps > 1:
                ghost["points"] = 2
                print(f"El fantasma en {ghost['pos']} se ha convertido en fantasma rojo.")
            # Si el fantasma está muerto, no hacer nada
            elif ghost["points"] <= 0:
                ghost["is_dead"] = True
                print(f"El fantasma en {ghost['pos']} ha sido eliminado.")

    def handle_portrait(self, portrait):
        # Lógica para manejar los retratos dentro del modelo
        if portrait["is_revealed"]:
            if portrait["is_fake"]:
                print(f"El retrato en {portrait['pos']} ha sido revelado como falso.")
            else:
                print(f"El retrato en {portrait['pos']} ha sido revelado correctamente.")
        if portrait["damaged"]:
            print(f"El retrato en {portrait['pos']} ha sido dañado.")


In [ ]:
GRID_SIZE = 10
AGENTS = 6
MAX_ENERGY = 120

# Inicializar model
model = MansionModel(GRID_SIZE, GRID_SIZE, AGENTS, MAX_ENERGY)

# Ejecutar modelo hasta que se pare solo
model.step()

# Mostrar cuántos turnos tomó
print("Total turns:", model.steps)

Total turns: 1


In [26]:
all_grid = model.datacollector.get_model_vars_dataframe()
all_grid.head(11)

,Grid,Steps
0,"[[[], [], [], [], [], [], [], [], [], []], [[]...",1
